In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("gpt-5") # "claude-sonnet-4-5"
standard_model = init_chat_model("gpt-5-nano")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [3]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [4]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

I didn’t water it today—I can’t reach the office plant myself. But I can help you stay on top of it.

Would you like me to set a recurring reminder to water it (e.g., weekly or on specific days) or add a note in our shared care log? If you tell me the plant type, I can pull up a more exact watering schedule. Quick tip in the meantime: water when the top inch of soil feels dry and make sure the pot has good drainage.


In [5]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [6]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

General rule: plan to repot when it’s rootbound—roots circling the pot, poking out of the drainage holes, the plant gets top‑heavy, water runs straight through, or the soil dries out in a day or two.

Timing: best in spring/early summer. Fast growers need it about every 12–18 months; slower houseplants every 2–3 years. Go up just 1–2 inches in pot diameter and refresh with fresh, well‑draining mix.

For ours, it looks fine right now. I’ll check the drainage holes and gently slide it out to peek at the roots this afternoon. If they’re tight, we’ll plan to repot next spring; if not, I’ll top up the soil and we can probably wait another year.


In [7]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-2025-08-07
